In [9]:
import numpy as np


In [12]:

class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        unique_classes = np.unique(y)

        # Stop if maximum depth is reached, or only one class remains
        if len(unique_classes) == 1 or (self.max_depth and depth >= self.max_depth):
            return unique_classes[0]

        # Variables to keep track of the best split
        best_gini = float('inf')
        best_split = None
        best_left_y = None
        best_right_y = None
        best_left_X = None
        best_right_X = None

        # Loop through each feature
        for feature in range(n_features):
            # Sort data based on the feature
            sorted_indices = X[:, feature].argsort()
            sorted_X = X[sorted_indices]
            sorted_y = y[sorted_indices]

            # Try splitting at each unique value in the sorted feature
            for i in range(1, n_samples):
                if sorted_X[i-1, feature] == sorted_X[i, feature]:
                    continue
                
                # Split the data
                left_X = sorted_X[:i]
                right_X = sorted_X[i:]
                left_y = sorted_y[:i]
                right_y = sorted_y[i:]

                # Calculate Gini Impurity for this split
                gini = self._gini_impurity(left_y, right_y)
                
                if gini < best_gini:
                    best_gini = gini
                    best_split = (feature, sorted_X[i-1, feature])
                    best_left_X = left_X
                    best_right_X = right_X
                    best_left_y = left_y
                    best_right_y = right_y

        # If no split improves the Gini, return the most frequent class
        if best_gini == float('inf'):
            return np.bincount(y).argmax()

        # Recursively build the tree on the best split
        left_tree = self._build_tree(best_left_X, best_left_y, depth + 1)
        right_tree = self._build_tree(best_right_X, best_right_y, depth + 1)

        # Return the decision node with feature, value, and the left and right subtrees
        return {
            'feature': best_split[0],
            'value': best_split[1],
            'left': left_tree,
            'right': right_tree
        }

    def _gini_impurity(self, left_y, right_y):
        left_size = len(left_y)
        right_size = len(right_y)
        total_size = left_size + right_size
        
        # Gini Impurity = 1 - sum(p_i^2)
        def gini(y):
            classes, counts = np.unique(y, return_counts=True)
            prob = counts / len(y)
            return 1 - np.sum(prob ** 2)
        
        gini_left = gini(left_y)
        gini_right = gini(right_y)

        # Weighted Gini Impurity of the split
        return (left_size / total_size) * gini_left + (right_size / total_size) * gini_right

    def predict(self, X):
        predictions = [self._predict_single(sample) for sample in X]
        return np.array(predictions)

    def _predict_single(self, sample):
        node = self.tree
        while isinstance(node, dict):
            if sample[node['feature']] <= node['value']:
                node = node['left']
            else:
                node = node['right']
        return node


In [18]:
X = np.array([[2, 3], [1, 2], [3, 3], [3, 4], [2, 1], [1, 1], [5, 3], [6, 5]])
y = np.array([0, 0, 0, 1, 1, 1, 0, 1])

dt = DecisionTree(max_depth=3)
dt.fit(X, y)

predictions = dt.predict(np.array([[3, 2], [5, 3]]))
print(predictions)  # Output: [0 0] (example)


[0 0]
